In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
from text3d2video.util import read_obj_with_uvs
from text3d2video.utilities.camera_placement import (
    cam_view_prompt,
    turntable_extrinsics,
)
from pytorch3d.renderer import FoVPerspectiveCameras

torch.set_grad_enabled(False)

In [50]:
# read mesh


device = "cuda"
mesh_path = "data/meshes/cat_statue.obj"
mesh, verts_uvs, faces_uvs = read_obj_with_uvs(mesh_path, device=device)

# setup cams
n_frames = 25
dist = 2.4
elev = 0
angles = np.linspace(0, 360, n_frames, endpoint=False)
R, T = turntable_extrinsics(dists=dist, angles=angles, elevs=elev)

frame_cams = FoVPerspectiveCameras(R=R, T=T, device="cuda", fov=30)
frame_meshes = mesh.extend(len(frame_cams))

# prompt suffixes
prompt_suffixes = [f", {cam_view_prompt(a, 0)} view" for a in angles]

  warnings.warn(f"Mtl file does not exist: {f}")



In [49]:
from text3d2video.artifacts.anim_artifact import AnimationArtifact

anim = AnimationArtifact.from_wandb_artifact_tag('rumba:latest')
frame_indices = anim.frame_indices(25)

frame_cams, frame_meshes = anim.load_frames(frame_indices)
verts_uvs, faces_uvs = anim.uv_data()

  meshes = torch.load(self._meshes_path())

  cams = torch.load(self._cams_path())

  meshes = torch.load(self._meshes_path())



In [51]:
from text3d2video.rendering import render_rgb_uv_map
from text3d2video.utilities.video_comparison import display_vids
from text3d2video.utilities.video_util import pil_frames_to_clip

name = "mv_cat_statue_20"
uvs = render_rgb_uv_map(frame_meshes, frame_cams, verts_uvs, faces_uvs)
vid = pil_frames_to_clip(uvs, fps=15)
display_vids([vid], title=name)

In [52]:
from text3d2video.artifacts.anim_artifact import AnimationArtifact

new_anim = AnimationArtifact.create_empty_artifact(name)
new_anim.write_frames(frame_cams, frame_meshes)
new_anim.write_uv_data(verts_uvs, faces_uvs)
new_anim.log_standalone()

wandb: Adding directory to artifact (/tmp/local_artifacts/animation/mv_cat_statue_20)... Done. 0.0s
